In [1]:
from train_latent_space import BRepDS, BRepFaceAutoencoder, BRepFaceEncoder, implicit_part_to_data, ImplicitPart, ImplicitDecoder

In [24]:
ipart = ImplicitPart('../../cube.x_t', 10, 100, True)

In [25]:
data = implicit_part_to_data(ipart, 10)

In [26]:
data

HetData(bounding_box=[2, 3], face_surfaces=[6, 5], face_surface_parameters=[6, 11], face_surface_flipped=[6], loop_types=[6, 10], loop_length=[6], edge_curves=[12, 3], edge_curve_parameters=[12, 11], edge_curve_flipped=[12], edge_length=[12], vertex_positions=[8, 3], face_to_face=[3, 12], face_to_loop=[2, 6], loop_to_edge=[2, 24], edge_to_vertex=[2, 24], loop_to_vertex=[2, 0], edge_to_vertex_is_start=[24], loop_to_edge_flipped=[24], surface_bounds=[6, 2, 2], surface_coords=[6, 10, 2], surface_samples=[6, 10, 7], curve_bounds=[12, 2], curve_samples=[12, 4, 6])

In [9]:
import torch
model = BRepFaceAutoencoder(64,1024,4)
ckpt = torch.load('../../BRepFaceAutoencoder_64_1024_4.ckpt')
model.load_state_dict(ckpt['state_dict'])

<All keys matched successfully>

In [12]:
n_faces = 6
N = 50
line = torch.linspace(-0.1,1.1,N)
grid = torch.cartesian_prod(line, line)
grids = grid.repeat(n_faces,1)
indices = torch.arange(n_faces).repeat_interleave(N*N, dim=0)

c:\Users\Ben\anaconda3\envs\automate\lib\site-packages\torch\functional.py:1069: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.cartesian_prod(tensors)  # type: ignore[attr-defined]


In [27]:
pred = model(data, grids, indices)

In [21]:
import meshplot as mp

In [30]:
mp.plot(pred[:,:3].detach().numpy(), c=pred[:,3].detach().numpy(), shading={'point_size':1})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.002272…

In [31]:
from chamfer_distance import ChamferDistance

ImportError: DLL load failed while importing cd: The specified module could not be found.

In [ ]:
chamdef